<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=160px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Graphs and Networks</h1>
<h1>Lesson I - Graph Representations</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint

import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: f4224b20bee304961526b45e7eb2fe1e30ac41e3

numpy     : 1.19.2
json      : 2.0.9
matplotlib: 3.3.2
watermark : 2.1.0



Load default figure style

In [3]:
plt.style.use('./d4sci.mplstyle')

## Edge List

Let us start by defining a list of edges. This will give us our first "dataset" to work with

In [4]:
edge_list = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'E'),
    ('B', 'C'),
    ('C', 'D'),
    ('C', 'E'),
    ('D', 'E')]

This is a particularly useful representation as many datasets are distributed in this (or a closely related) format. From this list, we can easily measure the number of edges that constitute our network. It's main limitations are that it has no way to explictly take into account disconnected nodes (it only accounts for nodes that are part of edges) and no indication on whether it is directed or not.

In [5]:
number_edges = len(edge_list)
print(number_edges)

7


To get the number of nodes, $k$, is a bit trickier. We must go edge by edge and keep track of all new nodes. For efficiency, we use a set to automatically remove duplicates

In [6]:
nodes = set()

for edge in edge_list:
    nodes.update(edge)
    
number_nodes = len(nodes)
print(number_nodes)

5


Now we know that we have 5 nodes and 7 edges in our network. The node ids are:

In [7]:
nodes

{'A', 'B', 'C', 'D', 'E'}

## Adjacency List

A closely related data structure to the edge list is the adjacency list. In this formulation, we simply use a dictionary to map each node to its set of neighbors

In [8]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set()
    
    adj_list[node_i].add(node_j)

Our adjaceny list is then:

In [9]:
pprint(adj_list)

{'A': {'E', 'C', 'B'}, 'B': {'C'}, 'C': {'D', 'E'}, 'D': {'E'}}


In this approach we inherently assumed that our network is directed (or, equivalently, that both edge directions are present in the data). To generate an undirected version we must make a simple modification to our code to manually add the opposite direction edge

In [10]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set() # 'set' is used to prevent accidental multiple edges
    
    adj_list[node_i].add(node_j)
    
    # Manually add the opposite direction edge
    if node_j not in adj_list:
        adj_list[node_j] = set()
    
    adj_list[node_j].add(node_i)

The undirected adjacency list represenation is then:

In [11]:
pprint(adj_list)

{'A': {'E', 'C', 'B'},
 'B': {'C', 'A'},
 'C': {'D', 'B', 'E', 'A'},
 'D': {'E', 'C'},
 'E': {'D', 'C', 'A'}}


## Adjacency Matrix

We now move on to generating an Adjacency Matrix view of the network. For this we must have two things: 

- the number of nodes in the network
- A mapping between the original node ids and a sequential numerical ID

We start by building out the numerical ID mapping. As we do, we get the number of nodes for free

In [12]:
node_id = {}
node_count = 0

for node_i, node_j in edge_list:
    if node_i not in node_id:
        node_id[node_i] = node_count
        node_count += 1
    
    # Make sure we have an id for both nodes
    # This is necessary, irregardless of whether the network is directed or undirected
    if node_j not in node_id:
        node_id[node_j] = node_count
        node_count += 1

We can check that each of the original node ids is correctly mapped to a sequential number

In [13]:
node_id

{'A': 0, 'B': 1, 'C': 2, 'E': 3, 'D': 4}

Finally, we are ready to build our adjacency matrix. We start by declaring the data structurewe will use

In [14]:
adj_matrix = np.zeros((node_count, node_count), dtype='int')

And we can now populate the matrix entries. For generality, we'll include a flag to control wether or not the graph is directed. As we don't have any weights associated with our edges, we assign to each of them a value of 1.

In [15]:
is_directed = False

for node_i, node_j in edge_list:    
    # Get the correct node ids
    node_i = node_id[node_i]
    node_j = node_id[node_j]
    
    adj_matrix[node_i, node_j] = 1 # Unweighted network

    if not is_directed:
        adj_matrix[node_j, node_i] = 1 # Undirected networks

Our Adjacency Matrix is then:

In [16]:
adj_matrix

array([[0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0]])

As we can see, the Adjacency matrix representation is very wasteful, using 25 values to store a 7 (14) edge network plus 5 dictionary entries for the id mappings. On the other hand, it has several mathematical advantages

## Adjacency Dict

The final graph representation we will explore is the Adjacency Dict. This is a generalization of the Adjacency List covered above that is a bit more flexible and is able to accuratly account for diconnected nodes, weights, etc. For this we will need two datastructures, one to store node information and one for edges. For the sake of flexbility, we will use dicts for both.

In [17]:
nodes = {}
edges = {}
is_directed = False

And we can now populate them from our original edge_list

In [18]:
for node_i, node_j in edge_list:
    if node_i not in nodes:
        nodes[node_i] = {}
        edges[node_i] = {}
        
    if node_j not in nodes:
        nodes[node_j] = {}
        
        if not is_directed:
            edges[node_j] = {}
    
    edges[node_i][node_j] = {}
    
    if not is_directed:
        edges[node_j][node_i] = {}

Our set of nodes is simply:

In [19]:
nodes

{'A': {}, 'B': {}, 'C': {}, 'E': {}, 'D': {}}

In [20]:
edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}},
 'D': {'C': {}, 'E': {}}}

Where we chose to use dictinaries to allow for the storage of node attributes. Further, our edges are now:

In [21]:
edges['A']['B']['weight0'] = 3
edges['A']['B']['color'] = 'blue'

In [22]:
edges

{'A': {'B': {'weight0': 3, 'color': 'blue'}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}},
 'D': {'C': {}, 'E': {}}}

Where we once again opted to associate a dictionary to each edge. This affords us the hability to associate edge information (such as weights, etc) to each node.

As we can see, this is the most flexible representation and can be easily converted to one of the other representations if necessary so we will use this approach for the rest of the lecture

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>